In [1]:
import numpy as np
import soundfile as sf
from scipy.signal import firwin, filtfilt
from tqdm import tqdm

def fir_bandpass(y, lowcut, highcut, sr, numtaps=1001 ):
    nyquist = 0.5 * sr
    taps = firwin(numtaps, [lowcut / nyquist, highcut / nyquist], pass_zero=False)
    filtered_signal = filtfilt(taps, [1.0], y)
    return filtered_signal

def get_random_time_interval(mid = 1, dev = 0.25):
    return np.random.rand() * (dev * 2) - dev + mid

def get_random_cutoff_frequency( sr ):
    frequencies = [np.random.choice( int(sr / 6) ), 
                   np.random.choice( np.arange(int(sr * 1.1 / 6), sr // 2) )  
                  ]
    min_frequency = min(frequencies)
    max_frequency = max(frequencies)
    return min_frequency, max_frequency

def get_white_noise( num_white_noise_blocks = 10, sr = 32000 ):
    label = []
    current_time = 0
    for _ in range(num_white_noise_blocks):
        onset = current_time + get_random_time_interval()
        offset = onset + get_random_time_interval()
        current_time = offset
        min_freq, max_freq = get_random_cutoff_frequency(sr)
        label.append( {
            "onset": onset,
            "offset": offset,
            "min_frequency": min_freq,
            "max_frequency": max_freq
        } )

    y = np.zeros( int((current_time + 1) * sr ) ).astype(np.float32)
    for item in tqdm(label):
        onset_idx = int( item["onset"] * sr )
        offset_idx = int( item["offset"] * sr )
        white_noise_signal = np.random.rand( offset_idx - onset_idx ).astype(np.float32)
        ## apply bandpass filtering
        white_noise_signal = fir_bandpass(white_noise_signal, 
                                          item["min_frequency"], 
                                          item["max_frequency"], 
                                          sr )
        y[ onset_idx:offset_idx ] = white_noise_signal
    return y.astype(np.float32), label

sr = 32000 
y, label = get_white_noise(sr = sr)
sf.write("white-noise.wav", y, samplerate = sr)
label

100%|█████████████████████████████████████████████████████| 10/10 [00:00<00:00, 29.85it/s]


[{'onset': 0.870058609969713,
  'offset': 2.0470729373394736,
  'min_frequency': 507,
  'max_frequency': 8550},
 {'onset': 3.2120809665701175,
  'offset': 4.160230387315875,
  'min_frequency': 1941,
  'max_frequency': 15262},
 {'onset': 5.13260362180458,
  'offset': 6.310577644113367,
  'min_frequency': 3405,
  'max_frequency': 10992},
 {'onset': 7.386949402246106,
  'offset': 8.470514547859267,
  'min_frequency': 4120,
  'max_frequency': 8831},
 {'onset': 9.459600601067086,
  'offset': 10.209840586993908,
  'min_frequency': 3496,
  'max_frequency': 12654},
 {'onset': 11.03539133522215,
  'offset': 12.237344743428856,
  'min_frequency': 3820,
  'max_frequency': 5871},
 {'onset': 13.373390337990154,
  'offset': 14.464489263449275,
  'min_frequency': 920,
  'max_frequency': 8066},
 {'onset': 15.353393000748945,
  'offset': 16.185389039692833,
  'min_frequency': 1949,
  'max_frequency': 5986},
 {'onset': 17.01547177336291,
  'offset': 18.16664729590825,
  'min_frequency': 1599,
  'max_fre